In [1]:
from flashtext import KeywordProcessor
import pandas as pd
from pathlib import Path
from collections import defaultdict
from IPython.display import display, HTML
from collections import OrderedDict
import networkx as nx
from ipywidgets import interact_manual, widgets, Layout

## Exploring WikiCSSH

In [2]:
%%time
cat2pages = pd.read_csv('../data/v1/WikiCSSH_category2page.csv').groupby("cat_title").page_title.agg(list)

Wall time: 1.97 s


In [3]:
def generated_WikiCSSH_graph():
    df_categories = pd.read_csv('../data/v1/WikiCSSH_categories.csv')
    df_category_links_all = pd.read_csv('../data/v1/WikiCSSH_category_links_all.csv')
#     df_category_links_all = pd.concat([
#         df_category_links,
#         df_categories[df_categories.category.isin(root_child_cats)].rename(columns={
#             "category": "child_cat", 
#             "level": "child_level",
#         }).assign(parent_cat="<ROOT>", parent_level=0),
#         pd.DataFrame({
#             "parent_cat": [""],
#             "child_cat": ["<ROOT>"],
#             "parent_level": [-1],
#             "child_level": [0],
#         })
#     ], axis=0, sort=True
#     ).sort_values(["parent_level", "child_level"])

    G = nx.DiGraph()
    G.add_edges_from(
        df_category_links_all
        .set_index(["parent_cat", "child_cat"])
        .to_dict(orient="index", into=OrderedDict)
    )
    return G

In [4]:
G = generated_WikiCSSH_graph()

In [5]:
def show_context_of_category(node):
    parents = list(G.predecessors(node))
    children = list(G.neighbors(node))
    table_header = "<thead><tr><th>Parents</th><th>Node</th><th>Children</th></tr></thead>"
    parents_list = "\n".join([f"<li><a href='https://en.wikipedia.org/wiki/Category:{i}'>{i}</a></li>" for i in parents])
    children_list = "\n".join([f"<li><a href='https://en.wikipedia.org/wiki/Category:{i}'>{i}</a></li>" for i in children])
    node_list = f"<li><a href='https://en.wikipedia.org/wiki/Category:{node}'>{node}</a></li>"
    page_list = " | ".join([
        f"<a href='https://en.wikipedia.org/wiki/{i}'>{i}</a>"
        for i in cat2pages[node]
    ])
    table_body = f"""<tbody>
    <tr>
        <td><ul>{parents_list}</ul></td>
        <td style='background-color: pink'>{node_list}</td>
        <td><ul>{children_list}</ul></td>
    </tr>
    <tr><td colspan='3'><strong>Pages</strong><br/>{page_list}</td></tr>
    </tbody>"""
    div = f"""
    <style>.wikicssh td, .wikicssh th {{text-align:left}}</style>
    <div class='wikicssh'>
    <table>{table_header}{table_body}</table>
    </div>"""
    return HTML(div)

In [6]:
show_context_of_category("Artificial_intelligence")

Parents,Node,Children
Areas_of_computer_science Unsolved_problems_in_computer_science,Artificial_intelligence,Fuzzy_logic Machine_learning Turing_tests Rule_engines Robots Computer_vision Logic_programming Open_source_artificial_intelligence Virtual_assistants Cloud_robotics Game_artificial_intelligence AI_accelerators Affective_computing Cognitive_architecture Artificial_intelligence_applications Artificial_immune_systems Robotics Artificial_Intelligence_existential_risk Evolutionary_computation


In [7]:
interact_manual(show_context_of_category, node="Computer_vision");

interactive(children=(Text(value='Computer_vision', description='node'), Button(description='Run Interact', st…

## Tagging text using WikiCSSH

In [8]:
%%time
def get_WikiCSSH_tagger():
    page2cats = (
        pd.read_csv('../data/v1/WikiCSSH_category2page.csv')
        .groupby("page_title")
        .cat_title
        .agg(lambda x: list(x))
        .to_dict()
    )
    
    processor = KeywordProcessor()
    # categories
    processor.add_keywords_from_dict(
        {
            f'Category:{k}': [f'{k.lower().replace("_", " ")}']
            for k in pd.read_csv("../data/v1/WikiCSSH_categories.csv").category.values
        }
    )

    for row in pd.read_csv('../data/v1/WikiCSSH_page2redirect.csv').values:
        if isinstance(row[-1], float):
            row[-1] = row[0]
        processor.add_keyword(row[-1].lower().replace("_", " "), row[0])
        
    return processor, page2cats

Wall time: 0 ns


In [9]:
%%time
processor, page2cats = get_WikiCSSH_tagger()

Wall time: 48.8 s


In [10]:
text = """In the last decade, we experienced an urgent need for a flexible, context-sensitive, fine-grained, and machine-actionable representation of scholarly knowledge and corresponding infrastructures for knowledge curation, publishing and processing. Such technical infrastructures are becoming increasingly popular in representing scholarly knowledge as structured, interlinked, and semantically rich Scientific Knowledge Graphs (SKG). Knowledge graphs are large networks of entities and relationships, usually expressed in W3C standards such as OWL and RDF. SKGs focus on the scholarly domain and describe the actors (e.g., authors, organizations), the documents (e.g., publications, patents), and the research knowledge (e.g., research topics, tasks, technologies) in this space as well as their reciprocal relationships. These resources provide substantial benefits to researchers, companies, and policymakers by powering several data-driven services for navigating, analysing, and making sense of research dynamics. Some examples include Microsoft Academic Graph (MAG), Open Academic Graph (combining MAG and AMiner), ScholarlyData, PID Graph, Open Research Knowledge Graph, OpenCitations, and OpenAIRE research graph. Current challenges in this area include: i) the design of ontologies able to conceptualise scholarly knowledge, ii) (semi-)automatic extraction of entities and concepts, integration of information from heterogeneous sources, identification of duplicates, finding connections between entities, and iii) the development of new services using this data, that allow to explore this information, measure research impact and accelerate science. This workshop aims at bringing together researchers and practitioners from different fields (including, but not limited to, Digital Libraries, Information Extraction, Machine Learning, Semantic Web, Knowledge Engineering, Natural Language Processing, Scholarly Communication, and Bibliometrics) in order to explore innovative solutions and ideas for the production and consumption of Scientific Knowledge Graphs (SKGs)."""

In [11]:
processor.extract_keywords(text, span_info=True)

[('Experience', 23, 34),
 ('Granularity', 85, 97),
 ('Scholarly_method', 140, 149),
 ('Knowledge', 150, 159),
 ('Knowledge', 198, 207),
 ('Scholarly_method', 326, 335),
 ('Knowledge', 336, 345),
 ('Semantics', 378, 390),
 ('Knowledge', 407, 416),
 ('Category:Graphs', 417, 423),
 ('Knowledge', 431, 440),
 ('Category:Graphs', 441, 447),
 ('Entity', 470, 478),
 ('World_Wide_Web_Consortium', 519, 532),
 ('Scholarly_method', 572, 581),
 ('Document', 649, 658),
 ('Research', 698, 706),
 ('Knowledge', 707, 716),
 ('Research', 724, 732),
 ('Category:Space', 770, 775),
 ('Research', 867, 878),
 ('Business', 880, 889),
 ('Research', 996, 1004),
 ('CONFIG.SYS', 1029, 1036),
 ('Microsoft_Academic', 1037, 1055),
 ('Academy_(educational_institution)', 1074, 1082),
 ('Open_research', 1143, 1156),
 ('Ontology_(information_science)', 1157, 1172),
 ('Research', 1202, 1210),
 ('Category:Area', 1245, 1249),
 ('CONFIG.SYS', 1250, 1257),
 ('Category:Design', 1266, 1272),
 ('Ontology', 1276, 1286),
 ('Concep

In [12]:
def get_html(text, processor):
    spans = processor.extract_keywords(text, span_info=True)
    prev = 0
    parts = []
    category_counts = defaultdict(int)
    for entity, start, end in spans:
        if entity.startswith("Category:"):
            entity_cats = [entity.replace("Category:", "")]
        else:
            entity_cats = [c for c in page2cats.get(entity, [])]
        for cat in entity_cats:
            category_counts[cat] += 1
        if start > prev:
            parts.append(text[prev:start])
        parts.append(f"<a href='https://en.wikipedia.org/wiki/{entity}' title='{entity}'>{text[start:end]}</a>")
        prev = end
    tagged_doc = "".join(parts).replace("\n", "<br/>")
    pred_categories = " | ".join([
        f"<a href='https://en.wikipedia.org/wiki/Category:{k}' title='{k}'>{k}</a> ({v})"
        for k,v in sorted(category_counts.items(), key=lambda x: x[1], reverse=True)
    ])
    final_div = f"""<div>
    <div>
        <h3>Tagged document:</h3>
        {tagged_doc}
    </div>
    <div>
        <h3>Predicted categories:</h3>
        {pred_categories}
    </div>
    </div>"""
    return HTML(final_div)
    

In [13]:
display(get_html(text, processor))

In [14]:
text_area_widget = widgets.Textarea(
    value=text,
    placeholder="Type your text hear",
    description='String:',
    disabled=False,
    layout=Layout(width="90%")
)
text_area_widget.rows=10;
interact_manual(lambda text: get_html(text, processor), text=text_area_widget);

interactive(children=(Textarea(value='In the last decade, we experienced an urgent need for a flexible, contex…